### Coleta e persistência dos dados neste notebook

**tipo de ingestão**: full load (recomendado para ingestão de dados inicial no datalake)

**origem**: Yahoo finance, Wikipedia, Fear and Greed (alternative.me)

**destino**: bronze/database/bitcoin/raw/btc.csv

**formato do data no data lake**: .csv


#### Instruções

1. Renomear o arquivo .env_exemplo para somente .env
2. Adicionar popular as variaveis conforme o padrão de nomenclatura que voce utilizar

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from transformers import pipeline
import yfinance as yf
import mwclient
import time
import requests
import json
from datetime import datetime
from statistics import mean
from io import StringIO
from dotenv import load_dotenv
import os

##carrega variaveis de ambiente
load_dotenv()

True

Extrai os dados de bitcoin do Yahoo Finance

In [2]:
def extract_btc(data_inicio: datetime) -> pd.DataFrame:
    ticker = yf.Ticker("BTC-USD")
    btc = ticker.history(start=data_inicio)
    return btc

Formata a base de bitcoin

In [3]:
def format_base(df:pd.DataFrame) -> pd.DataFrame:
    df.index = pd.to_datetime(df.index).tz_localize(None)
    del df["Dividends"]
    del df["Stock Splits"]
    df.columns = [c.lower() for c in df.columns]
    return df

Adiciona as colunas Target e Tomorrow

In [4]:
def target_and_clean(df: pd.DataFrame) -> pd.DataFrame:
    df["tomorrow"] = df["close"].shift(-1)
    df["target"] = (df["tomorrow"] > df["close"]).astype(int)
    df.dropna(inplace=True)
    return df

In [5]:
data_inicio = datetime.strptime('2018-01-01', '%Y-%m-%d')
data_fim = datetime.strptime('2024-08-31', '%Y-%m-%d')
btc = format_base(extract_btc(data_inicio))
btc =  target_and_clean(btc)
btc.head()

,open,high,low,close,volume,tomorrow,target
Date,,,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,10291200000,14982.099609,1
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,16846600192,15201.000000,1
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,16871900160,15599.200195,1
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,21783199744,17429.500000,1
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,23840899072,17527.000000,1


Extrai os dados das reviews de bitcoin da wikipedia

In [6]:
def extract_reviews() -> list:
   site = mwclient.Site("en.wikipedia.org")
   page = site.pages["Bitcoin"]
   revs = list(page.revisions(start=data_inicio, dir="newer"))
   revs = sorted(revs, key=lambda rev: rev["timestamp"])
   return revs

Classifica o sentimento relacionado as reviews

seleciona o modelo utilizado para classificação das reviews

In [7]:
sentiment_pipeline = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\brisa\OneDrive\Documentos\btc\bitcoin_trend_prediction\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


transforma o score das classificações negativas

In [8]:
def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

cria a base de sentimentos

In [9]:
edits = {}
revs = extract_reviews()

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

prepara e limpa a base

In [10]:
for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [11]:
def create_edits_df(edits: dict) -> pd.DataFrame:
    edits_df = pd.DataFrame.from_dict(edits, orient="index")
    edits_df.index = pd.to_datetime(edits_df.index)
    return edits_df

edits_df = create_edits_df(edits)

Ajusta as competencias das reviews de sentimento e cria uma janela de média móvel

In [12]:
def improve_edits_df(edits_df: dict) -> pd.DataFrame:
    dates = pd.date_range(start=data_inicio, end=data_fim)
    edits_df = edits_df.reindex(dates, fill_value=0)
    edits_df["edit_count"] = edits_df["edit_count"].shift(1)
    edits_df["sentiment"] = edits_df["sentiment"].shift(1)
    edits_df["neg_sentiment"] = edits_df["neg_sentiment"].shift(1)
    edits_df = edits_df.dropna()
    rolling_edits = edits_df.rolling(30, min_periods=30).mean()
    rolling_edits = rolling_edits.dropna()
    return rolling_edits

rolling_edits = improve_edits_df(edits_df)
rolling_edits.head()

,edit_count,sentiment,neg_sentiment
2018-01-31,6.466667,-0.347147,0.607268
2018-02-01,6.433333,-0.380104,0.607268
2018-02-02,6.233333,-0.382591,0.590601
2018-02-03,5.566667,-0.406722,0.601712
2018-02-04,5.266667,-0.397372,0.597167


Une as bases de bitcoin e sentimentos

In [13]:
btc = btc.merge(rolling_edits, left_index=True, right_index=True)
btc.head()

,open,high,low,close,volume,tomorrow,target,edit_count,sentiment,neg_sentiment
2018-01-31,10108.200195,10381.599609,9777.419922,10221.099609,8041160192,9170.540039,0,6.466667,-0.347147,0.607268
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9959400448,8830.750000,0,6.433333,-0.380104,0.607268
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712,9174.910156,1,6.233333,-0.382591,0.590601
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080,8277.009766,0,5.566667,-0.406722,0.601712
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824,6955.270020,0,5.266667,-0.397372,0.597167


Cria colunas adicionais ao dataframe existente para verificar possíveis tendências em horizontes de tempo

In [14]:
def trends_col(df: pd.DataFrame) -> pd.DataFrame:
    horizons = [2, 7, 30, 60, 365]

    for horizon in horizons:
        rolling_averages = df.rolling(horizon, min_periods=1).mean()

        ratio_column = f'close_ratio_{horizon}'
        df[ratio_column] = df['close'] / rolling_averages['close']

        edit_column = f'edit_{horizon}'
        df[edit_column] = rolling_averages['edit_count']

        rolling = df.rolling(horizon, closed='left', min_periods=1).mean()
        trend_column = f'trend_{horizon}'
        df[trend_column] = rolling['target']

    return df
btc = trends_col(btc)

Extrai a base de Fear and Greed da Alternative.me

In [15]:
def collect_fear_greed(api_url:str, limit:int, drop_colls:list, rename_colls:list) -> pd.DataFrame:
    response = requests.get(f"{api_url}?limit={limit}")
    n_data = response.json()
    df = pd.read_json(StringIO(json.dumps(n_data["data"])))
    df.drop(columns=drop_colls, axis=1, inplace=True)
    df.rename(columns=rename_colls, inplace=True)
    df.set_index("date", inplace=True)
    return df

In [16]:
api_url = "https://api.alternative.me/fng/"
colls_to_drop = ["time_until_update"]
coll_names = {"value":"fng_index", 
              "value_classification":"fng_class",
              "timestamp": "date"}

data_fg = collect_fear_greed(api_url=api_url, limit=0, drop_colls=colls_to_drop, rename_colls=coll_names)

In [17]:
data_fg = data_fg.sort_index()

Ajusta as competencias dos dados de fear and greed

In [18]:
def improve_data_fg(df: pd.DataFrame) -> pd.DataFrame:
    df["fng_index"] = df["fng_index"].shift(1)
    df["fng_class"] = df["fng_class"].shift(1)
    df.dropna(inplace=True)
    return df

data_fg = improve_data_fg(data_fg)
data_fg.head()

,fng_index,fng_class
date,,
2018-02-02,30.0,Fear
2018-02-03,15.0,Extreme Fear
2018-02-04,40.0,Fear
2018-02-05,24.0,Extreme Fear
2018-02-06,11.0,Extreme Fear


Une as bases de bitcoin, sentimentos e fear and greed

In [19]:
btc = btc.merge(data_fg, left_index=True, right_index=True)
btc.head()

,open,high,low,close,volume,tomorrow,target,edit_count,sentiment,neg_sentiment,...,edit_30,trend_30,close_ratio_60,edit_60,trend_60,close_ratio_365,edit_365,trend_365,fng_index,fng_class
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712,9174.910156,1,6.233333,-0.382591,0.590601,...,6.377778,0.000000,0.938696,6.377778,0.000000,0.938696,6.377778,0.000000,30.0,Fear
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080,8277.009766,0,5.566667,-0.406722,0.601712,...,6.175000,0.333333,0.981345,6.175000,0.333333,0.981345,6.175000,0.333333,15.0,Extreme Fear
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824,6955.270020,0,5.266667,-0.397372,0.597167,...,5.993333,0.250000,0.906090,5.993333,0.250000,0.906090,5.993333,0.250000,40.0,Fear
2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,9285289984,7754.000000,1,4.733333,-0.385526,0.592116,...,5.783333,0.200000,0.792931,5.783333,0.200000,0.792931,5.783333,0.200000,24.0,Extreme Fear
2018-02-06,7051.750000,7850.700195,6048.259766,7754.000000,13999800320,7621.299805,0,4.700000,-0.385761,0.592116,...,5.628571,0.333333,0.898887,5.628571,0.333333,0.898887,5.628571,0.333333,11.0,Extreme Fear


SUGESTÃO: se for trabalhar nos dados a partir deste ponto, descomente a celula abaixo para gerar um backup do dataframe para agilizar seu trabalho

In [20]:
# work_backup = btc.copy()
# work_backup.head()

Organiza as colunas

In [21]:
sequencia = [
       'open', 'high', 'low', 'close', 'volume', 'tomorrow',
       'edit_count', 'sentiment', 'neg_sentiment', 'close_ratio_2', 'edit_2',
       'trend_2', 'close_ratio_7', 'edit_7', 'trend_7', 'close_ratio_30',
       'edit_30', 'trend_30', 'close_ratio_60', 'edit_60', 'trend_60',
       'close_ratio_365', 'edit_365', 'trend_365', 'fng_index', 'fng_class',
       'target'
]
btc = btc[sequencia]
btc.head()

,open,high,low,close,volume,tomorrow,edit_count,sentiment,neg_sentiment,close_ratio_2,...,trend_30,close_ratio_60,edit_60,trend_60,close_ratio_365,edit_365,trend_365,fng_index,fng_class,target
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712,9174.910156,6.233333,-0.382591,0.590601,0.981124,...,0.000000,0.938696,6.377778,0.000000,0.938696,6.377778,0.000000,30.0,Fear,1
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080,8277.009766,5.566667,-0.406722,0.601712,1.019114,...,0.333333,0.981345,6.175000,0.333333,0.981345,6.175000,0.333333,15.0,Extreme Fear,0
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824,6955.270020,5.266667,-0.397372,0.597167,0.948550,...,0.250000,0.906090,5.993333,0.250000,0.906090,5.993333,0.250000,40.0,Fear,0
2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,9285289984,7754.000000,4.733333,-0.385526,0.592116,0.913228,...,0.200000,0.792931,5.783333,0.200000,0.792931,5.783333,0.200000,24.0,Extreme Fear,1
2018-02-06,7051.750000,7850.700195,6048.259766,7754.000000,13999800320,7621.299805,4.700000,-0.385761,0.592116,1.054301,...,0.333333,0.898887,5.628571,0.333333,0.898887,5.628571,0.333333,11.0,Extreme Fear,0


Realiza a ingestão do dataframe no data lake

In [25]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [26]:
# Converte o dataframe em uma string CSV
csv_buffer = StringIO()
btc.to_csv(csv_buffer, index=True)

In [27]:
bucket_name = os.environ['BUCKET_NAME']
csv_file_name = os.environ['CSV_FILE_NAME']
bucket_layer = os.environ['BUCKET_LAYER']
object_name = f'{bucket_layer}/{csv_file_name}'

In [28]:
def upload_to_s3(bucket_name, object_name, csv_buffer):
    try:
        s3_resource = boto3.resource('s3')
        req_metadata = s3_resource.Object(bucket_name, object_name).put(Body=csv_buffer.getvalue())
        status = req_metadata['ResponseMetadata']['HTTPStatusCode']
        if status == 200:
            print('Upload sucessful')
        else:
            print(f'Upload failed, status {status}')
    except ClientError as e:
        logging.error(e)

upload_to_s3(bucket_name, object_name, csv_buffer)

Upload sucessful


In [27]:
# baixa csv localmente
#btc.to_csv('btc.csv')